### Search Engine With Tools and Agents

In [8]:
## In built tools
## Arxiv--Website with research papers
## Tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun #this interacts with the tool itself
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper #to interact with the tools we need a wrapper


In [9]:
#using the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [10]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)


arxiv


In [11]:
#Combining these two tools
#combine them in the form of list
tools=[wiki,arxiv]


In [12]:
## Custom tools[RAG Tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [13]:
#I will load a website page and create that into a vectorstore and use it like my tool
loader=WebBaseLoader("https://docs.langchain.com/langsmith/home")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AFAA5D1F30>, search_kwargs={})

In [34]:
# Define a simple callable tool
def langsmith_search(query: str) -> str:
    """Search for LangSmith documentation using the FAISS retriever."""
    docs = retriever.get_relevant_documents(query)  # safe method
    return "\n\n".join([doc.page_content for doc in docs])

tools = [
    {
        "name": "wiki_search",
        "description": "Search Wikipedia",
        "func": lambda query: wiki.run(query)  # assuming wiki.run() works
    },
    {
        "name": "arxiv_search",
        "description": "Search Arxiv research papers",
        "func": lambda query: arxiv.run(query)  # assuming arxiv.run() works
    },
    {
        "name": "langsmith_search",
        "description": "Search LangSmith documentation",
        "func": langsmith_search
    }
]



In [35]:
tools=[wiki,arxiv,retriever_tool]

In [36]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\HP\\anaconda3\\envs\\genaiprojects\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for information about LangSmith documentation', func=<function <lambda> at 0x000001AFA925F6D0>)]

In [37]:
#Run all this tools with Agents and LLM Models
from langchain_groq import ChatGroq
import streamlit as st
#Tools, LLM-->AgentExecutor
groq_api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.3-70b-versatile")

In [38]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

# Create agent using LangGraph
agent = create_react_agent(
    model=llm,     # your ChatGroq model
    tools=tools
)

# Run agent
response = agent.invoke(
    {"messages": [HumanMessage(content="Tell me about LangSmith")]}
)

print(response["messages"][-1].content)


C:\Users\HP\AppData\Local\Temp\ipykernel_23816\868306958.py:5: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


LangSmith is a framework-agnostic platform for developing, debugging, and deploying AI agents and LLM applications. It helps you trace requests, evaluate outputs, test prompts, and manage deployments in one place. LangSmith works with any agent stack, whether you’re using an existing framework or building your own. It provides a visual interface to design, test, and refine applications end-to-end, and it meets the highest standards of data security and privacy with HIPAA, SOC 2 Type 2, and GDPR compliance.


from langchain import hub
prompt=hub.pull("") #Agent from hub
prompt.messages

##### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

##### Agent Executor

from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=true)
agent_executor.invoke({"input":"Tell me about Langmsmith"})


